In [ ]:
import React from 'react';
import { motion } from 'framer-motion';
import { format, subDays, startOfDay, isSameDay } from 'date-fns';
import { AreaChart, Area, XAxis, YAxis, Tooltip, ResponsiveContainer } from 'recharts';
import { TrendingUp, TrendingDown, Minus } from 'lucide-react';

export default function WeeklyChart({ scans = [], targets }) {
  const calorieTarget = targets?.daily_calorie_target || 2000;
  
  // Get last 7 days data
  const last7Days = Array.from({ length: 7 }, (_, i) => {
    const date = startOfDay(subDays(new Date(), 6 - i));
    const dayScans = (scans || []).filter(scan => 
      isSameDay(new Date(scan.created_date), date)
    );
    const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);
    const totalProtein = dayScans.reduce((sum, s) => sum + (s.protein || 0), 0);
    
    return {
      date: format(date, 'EEE'),
      fullDate: format(date, 'MMM d'),
      calories: totalCalories,
      protein: totalProtein,
      target: calorieTarget,
      mealsCount: dayScans.length
    };
  });

  const avgCalories = Math.round(
    last7Days.reduce((sum, d) => sum + d.calories, 0) / 7
  );
  
  const trend = last7Days[6].calories - last7Days[0].calories;
  const TrendIcon = trend > 100 ? TrendingUp : trend < -100 ? TrendingDown : Minus;
  const trendColor = Math.abs(avgCalories - calorieTarget) < 200 ? 'text-emerald-500' : 'text-amber-500';

  const CustomTooltip = ({ active, payload, label }) => {
    if (active && payload && payload.length) {
      return (
        <div className="bg-white rounded-xl p-3 shadow-lg border border-slate-100">
          <p className="text-sm font-medium text-slate-800 mb-1">{payload[0]?.payload?.fullDate}</p>
          <div className="space-y-1">
            <p className="text-sm text-slate-600">
              <span className="font-semibold text-orange-500">{payload[0]?.value}</span> kcal
            </p>
            <p className="text-xs text-slate-400">
              {payload[0]?.payload?.mealsCount} meals logged
            </p>
          </div>
        </div>
      );
    }
    return null;
  };

  return (
    <motion.div 
      initial={{ opacity: 0, y: 20 }}
      animate={{ opacity: 1, y: 0 }}
      className="bg-white rounded-3xl p-6 shadow-sm border border-slate-100"
    >
      <div className="flex items-start justify-between mb-6">
        <div>
          <h3 className="text-lg font-semibold text-slate-800">Weekly Overview</h3>
          <p className="text-sm text-slate-500 mt-1">Your calorie intake trend</p>
        </div>
        <div className="text-right">
          <div className="flex items-center gap-1 justify-end">
            <TrendIcon className={`w-4 h-4 ${trendColor}`} />
            <span className="text-sm font-medium text-slate-600">Avg</span>
          </div>
          <p className="text-xl font-bold text-slate-800">{avgCalories} <span className="text-sm font-normal text-slate-400">kcal/day</span></p>
        </div>
      </div>

      <div className="h-48">
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={last7Days} margin={{ top: 10, right: 10, left: -20, bottom: 0 }}>
            <defs>
              <linearGradient id="calorieGradient" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor="#f97316" stopOpacity={0.3}/>
                <stop offset="95%" stopColor="#f97316" stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <YAxis 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <Tooltip content={<CustomTooltip />} />
            <Area
              type="monotone"
              dataKey="calories"
              stroke="#f97316"
              strokeWidth={3}
              fill="url(#calorieGradient)"
            />
          </AreaChart>
        </ResponsiveContainer>
      </div>

      {/* Quick Stats */}
      <div className="grid grid-cols-3 gap-4 mt-6 pt-6 border-t border-slate-100">
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {last7Days.reduce((sum, d) => sum + d.mealsCount, 0)}
          </p>
          <p className="text-xs text-slate-500">Meals Logged</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-emerald-500">
            {last7Days.filter(d => Math.abs(d.calories - calorieTarget) < 200).length}
          </p>
          <p className="text-xs text-slate-500">Days On Target</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {Math.round(last7Days.reduce((sum, d) => sum + d.protein, 0) / 7)}g
          </p>
          <p className="text-xs text-slate-500">Avg Protein</p>
        </div>
      </div>
    </motion.div>
  );
}